In [0]:
from pyspark.sql.functions import array, to_timestamp, concat, lit, regexp_replace, col, when


bucket_name = "user-c1b2415b9314-bucket"

df_geo = spark.read.json(f"s3a://user-c1b2415b9314-bucket/topics/c1b2415b9314.geo/partition=0/")

df_user = spark.read.json(f"s3a://user-c1b2415b9314-bucket/topics/c1b2415b9314.user/partition=0/")

df_pin = spark.read.json(f"s3a://user-c1b2415b9314-bucket/topics/c1b2415b9314.pin/partition=0/")

In [0]:
# Cleaning df_user
# Creating user_name column by concatenating first_name and last_name
df_user = df_user.withColumn("user_name", concat("first_name", lit(" "), "last_name"))
# Dropping first_name and last_name columns
df_user = df_user.drop("first_name", "last_name")
# changing the date_joined column to a timestamp type
df_user = df_user.withColumn("date_joined", to_timestamp("date_joined"))
# Renaming the index column to ind
df_user = df_user.withColumnRenamed("index", "ind")
# Reordering the columns
df_user = df_user.select("ind", "user_name", "age", "date_joined")


df_user.printSchema()


In [0]:
# cleaning df_pin
df_pin = df_pin.replace({"User Info Error": None})
df_pin = df_pin.replace({"No description available": None})
df_pin = df_pin.replace({"No description available Story format": None})
df_pin = df_pin.replace({"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e": None})
df_pin = df_pin.replace({"No Title Data Available": None})
df_pin = df_pin.replace({"Image src error.": None})
# Amending "save_location" to show only the filepath
df_pin = df_pin.withColumn("save_location", regexp_replace("save_location", "Local save in ", ""))
# Replaces the k and M with the appropriate number of zeros and casts the follower_count column to int
df_pin = df_pin.withColumn(
    "follower_count",
    when(col("follower_count").endswith("k"), 
         regexp_replace(col("follower_count"), "k", "").cast("int") * 1000)
    .when(col("follower_count").endswith("M"), 
         regexp_replace(col("follower_count"), "M", "").cast("int") * 1000000)
    .otherwise(col("follower_count").cast("int"))
)
# Renamed the index column to ind
df_pin = df_pin.withColumnRenamed("index", "ind")
# Casted these two columns to int since they were originally long numerical type which wasn't neccesary for these smaller numbers (although in a real life situation i imagine it would be better to keep them as long)

df_pin = df_pin.withColumn("ind", df_pin["ind"].cast("int"))
# Reordering Columns (seems downloaded column is being dropped aswell?!!?!?!?!?!?!?!)
df_pin = df_pin.select("ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list", "is_image_or_video", "image_src", "save_location", "category")

df_pin.printSchema()



In [0]:
# cleaning df_geo

# making a coordinates column from longitude and latitude columns using an array
df_geo = df_geo.withColumn("coordinates", array("latitude", "longitude"))
df_geo = df_geo.drop("latitude", "longitude")
# changing the timestamp column to a timestamp type
df_geo = df_geo.withColumn("timestamp", to_timestamp("timestamp"))
# Renaming the index column to ind
df_geo = df_geo.withColumnRenamed("index", "ind")
# Reordering the columns
df_geo = df_geo.select("ind", "country", "coordinates", "timestamp")




df_geo.printSchema()


In [0]:
# Register as a temporary view
df_pin.createOrReplaceTempView("pin_table")
df_geo.createOrReplaceTempView("geo_table")
df_user.createOrReplaceTempView("user_table")


In [0]:
%sql
WITH ranked_categories AS (
    SELECT 
        g.country,
        p.category,
        COUNT(p.category) AS category_count,
        ROW_NUMBER() OVER (PARTITION BY g.country ORDER BY COUNT(p.category) DESC) AS rank
    FROM
        geo_table g
    JOIN
        pin_table p ON g.ind = p.ind
    GROUP BY
        g.country, p.category
)
SELECT
    country,
    category,
    category_count
FROM
    ranked_categories
WHERE
    rank = 1
ORDER BY
    country;


In [0]:
%sql
WITH ranked_categories AS (
    SELECT
        YEAR(g.timestamp) AS post_year,
        p.category,
        COUNT(p.category) AS category_count,
        ROW_NUMBER() OVER (PARTITION BY YEAR(g.timestamp) ORDER BY COUNT(p.category) DESC) AS rank
    FROM  
        pin_table p
    JOIN
        geo_table g ON p.ind = g.ind
    WHERE 
        YEAR(g.timestamp) BETWEEN 2018 AND 2022
    GROUP BY
        p.category, YEAR(g.timestamp)
)
SELECT
    post_year,
    category,
    category_count
FROM
    ranked_categories
WHERE
    rank = 1
ORDER BY
    post_year;


In [0]:
%sql
-- step one
WITH ranked_followers AS (
    SELECT
        g.country,
        p.poster_name,
        p.follower_count,
        ROW_NUMBER() OVER (PARTITION BY g.country ORDER BY p.follower_count DESC) AS rank
    FROM
        geo_table g
    JOIN 
        pin_table p ON g.ind = p.ind
)
SELECT
    country,
    poster_name,
    follower_count
FROM 
    ranked_followers
WHERE 
    rank = 1
ORDER BY
    follower_count DESC




In [0]:
%sql
-- step two

SELECT
    g.country,
    p.follower_count
FROM
    geo_table g
JOIN
    pin_table p ON g.ind = p.ind
ORDER BY  
    p.follower_count DESC
LIMIT 
    1


In [0]:
%sql
-- step two alternative method
WITH ranked_followers AS (
    SELECT
        g.country,
        p.poster_name,
        p.follower_count,
        ROW_NUMBER() OVER (PARTITION BY g.country ORDER BY p.follower_count DESC) AS rank
    FROM
        geo_table g
    JOIN 
        pin_table p ON g.ind = p.ind
)
SELECT
    country,
    follower_count
FROM 
    ranked_followers
WHERE 
    rank = 1
ORDER BY
    follower_count DESC
LIMIT
    1

In [0]:
%sql
WITH age_ranges AS (
    SELECT
        CASE
            WHEN age >= 18 AND age <= 24 THEN '18-24'
            WHEN age >= 25 AND age <= 35 THEN '25-35'
            WHEN age >= 36 AND age <= 49 THEN '36-49'
            WHEN age > 50 THEN '50+'
        END AS age_group,
        p.category
    FROM user_table u
    JOIN pin_table p ON u.ind = p.ind
),
category_counts AS (
    SELECT
        age_group,
        category,
        COUNT(*) AS category_count
    FROM age_ranges
    GROUP BY age_group, category
),
ranked_categories AS (
    SELECT
        age_group,
        category,
        category_count,
        ROW_NUMBER() OVER (PARTITION BY age_group ORDER BY category_count DESC) AS rank
    FROM category_counts
)
SELECT
    age_group,
    category,
    category_count
FROM
    ranked_categories
WHERE
    rank = 1
ORDER BY
    age_group;


In [0]:
%sql
WITH age_ranges AS (
    SELECT
        CASE
            WHEN age >= 18 AND age <= 24 THEN '18-24'
            WHEN age >= 25 AND age <= 35 THEN '25-35'
            WHEN age >= 36 AND age <= 49 THEN '36-49'
            WHEN age > 50 THEN '50+'
        END AS age_group,
        p.follower_count
    FROM user_table u
    JOIN pin_table p ON u.ind = p.ind
),
ranked_follower_count AS (
    SELECT
        age_group,
        follower_count,
        ROW_NUMBER() OVER (PARTITION BY age_group ORDER BY follower_count) AS row_num,
        COUNT(*) OVER (PARTITION BY age_group) AS total_rows
    FROM age_ranges
),
median_follower_calculator AS (
    SELECT
        age_group,
        CASE
            WHEN total_rows % 2 = 1 THEN -- Odd number of rows
                MAX(CASE WHEN row_num = (total_rows + 1) / 2 THEN follower_count END)
            ELSE -- Even number of rows
                AVG(CASE WHEN row_num IN (total_rows / 2, total_rows / 2 + 1) THEN follower_count END)
        END AS median_follower_count
    FROM
        ranked_follower_count
    GROUP BY
        age_group,
        total_rows
)
SELECT
    age_group,
    median_follower_count
FROM
    median_follower_calculator
ORDER BY
    age_group;  


In [0]:
%sql

SELECT
    YEAR(date_joined) AS post_year,
    COUNT(YEAR(date_joined)) AS number_users_joined
FROM
    user_table 
GROUP BY
    post_year

In [0]:
%sql

WITH new_users AS (
    SELECT
        YEAR(date_joined) AS post_year,
        COUNT(YEAR(date_joined)) AS number_users_joined,
        ind
    FROM
        user_table 
    GROUP BY
        post_year,
        ind
), ranked_follower_count AS (
    SELECT
        n.post_year,
        p.follower_count,
        ROW_NUMBER() OVER (PARTITION BY n.post_year ORDER BY p.follower_count) AS row_num,
        COUNT(*) OVER (PARTITION BY n.post_year) AS total_rows
    FROM 
        new_users n
    JOIN
        pin_table p ON n.ind = p.ind
), median_follower_calculator AS (
    SELECT
        post_year,
        CASE
            WHEN total_rows % 2 = 1 THEN -- Odd number of rows
                MAX(CASE WHEN row_num = (total_rows + 1) / 2 THEN follower_count END)
            ELSE -- Even number of rows
                AVG(CASE WHEN row_num IN (total_rows / 2, total_rows / 2 + 1) THEN follower_count END)
        END AS median_follower_count
    FROM
        ranked_follower_count
    GROUP BY
        post_year,
        total_rows
)
SELECT
    post_year,
    median_follower_count
FROM
    median_follower_calculator



In [0]:
%sql
WITH new_users AS (
    SELECT
        YEAR(date_joined) AS post_year,
        ind,
        age
    FROM
        user_table
), 
age_ranges AS (
    SELECT
        CASE
            WHEN age >= 18 AND age <= 24 THEN '18-24'
            WHEN age >= 25 AND age <= 35 THEN '25-35'
            WHEN age >= 36 AND age <= 49 THEN '36-49'
            WHEN age > 50 THEN '50+'
        END AS age_group,
        p.follower_count,
        n.post_year
    FROM 
        new_users n
    JOIN 
        pin_table p ON n.ind = p.ind
),
ranked_follower_count AS (
    SELECT
        age_group,
        follower_count,
        ROW_NUMBER() OVER (PARTITION BY post_year, age_group ORDER BY follower_count) AS row_num,
        COUNT(*) OVER (PARTITION BY post_year, age_group) AS total_rows,
        post_year
    FROM 
        age_ranges
),
median_follower_calculator AS (
    SELECT
        post_year,
        age_group,
        CASE
            WHEN total_rows % 2 = 1 THEN -- Odd number of rows
                MAX(CASE WHEN row_num = (total_rows + 1) / 2 THEN follower_count END)
            ELSE -- Even number of rows
                AVG(CASE WHEN row_num IN (total_rows / 2, total_rows / 2 + 1) THEN follower_count END)
        END AS median_follower_count
    FROM 
        ranked_follower_count
    GROUP BY 
        post_year,
        age_group,
        total_rows
)
SELECT
    age_group,
    post_year,
    median_follower_count
FROM 
    median_follower_calculator
ORDER BY 
    post_year, 
    age_group;


In [0]:
spark.conf.get("spark.databricks.clusterUsageTags.clusterId")